# STEP1.

In [48]:
class Variable:
  def __init__(self, data, name=None):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')

    self.data = data  # 통상값
    self.name = name # 변수들을 서로 구분하기 위해 name 인스턴스 변수 추가
    self.grad = None  # 미분값(s6에서 역전파를 이용한 미분 구현을 위해 Variable class 확장)
    self.creator = None  # 인스턴스 변수 추가 (s7)
    self.generation = 0  # 세대 수를 기록하는 변수 (s16)
    
  def set_creator(self, func):  # creator를 설정할 수 있도록 메서드 추가 (s7)
    self.creator = func
    self.generation = func.generation + 1  # 세대를 기록한다(부모 세대 + 1)(s16)

  def backward(self, retain_grad=False):
    if self.grad is None:
      self.grad = np.ones_like(self.data) # 역전파 구현시마다 y.grad = np.array(1.) 부분을 쓰던 것을 생략해도 되게끔 개선

    funcs = []
    seen_set = set()

    def add_func(f):  # func.append() 대신 메소드 정의 -> 리스트를 세대 순으로 정렬하는 역할 -> funcs.pop()은 자동으로 세대가 가장 큰 Dezero 함수를 꺼내게 됨.
      if f not in seen_set:
        funcs.append(f)
        seen_set.add(f)
        funcs.sort(key=lambda x: x.generation)

    add_func(self.creator)

    while funcs:
      f = funcs.pop()  # 함수를 가져온다
      gys = [output().grad for output in f.outputs] # 약한 참조로 변경: output()
      gxs = f.backward(*gys) # 인수에 별표를 붙여 역전파 호출 -> 리스트 언팩
      if not isinstance(gxs, tuple): # gxs가 튜플이 아니면 튜플로 변환
        gxs = (gxs,)

      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:  # 처음 설정 시
          x.grad = gx
        else:
          x.grad = x.grad + gx  # 그 이후부터는 미분값에 합산 (in-place 연산 쓰지 마!)

        if x.creator is not None:
          add_func(x.creator)  # func.appen(x.creator)에서 변경
      
      if not retain_grad:
        for y in f.outputs:
          y().grad = None  # y()는 약한 참조(weakref). 말단 변수 외에는 미분값을 유지하지 않음(메모리에서 삭제)


  def cleargrad(self):
    self.grad = None
  
  @property
  def shape(self):
    return self.data.shape
  
  @property
  def ndim(self):
    return self.data.ndim

  @property
  def size(self):
    return self.data.size

  @property
  def dtype(self):
    return self.data.dtype

  def __len__(self):
    return len(self.data)

  def __repr__(self):
    if self.data is None:
      return 'variable(None)'
    p = str(self.data).replace('\n', '\n' + ' ' * 9) # 2차원 array 행에 맞춰서 들여쓰기
    return 'variable(' + p + ')'

"""         
      x, y = f.input, f.output  # 함수의 입력과 출력을 가져온다.
      x.grad = f.backward(y.grad)  # backward 메서드를 호출한다.

      if x.creator is not None:
        funcs.append(x.creator)  # 하나 앞의 함수를 리스트에 추가한다.
"""

'         \n      x, y = f.input, f.output  # 함수의 입력과 출력을 가져온다.\n      x.grad = f.backward(y.grad)  # backward 메서드를 호출한다.\n\n      if x.creator is not None:\n        funcs.append(x.creator)  # 하나 앞의 함수를 리스트에 추가한다.\n'

In [2]:
import numpy as np

data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


In [3]:
data = np.array(2.0)  # x는 데이터를 담은 상자(인스턴스)이기 때문에, 인스턴스 변수(x.data)가 아닌, 일반 변수 data에 새로운 값 할당해도 인스턴스(객체) 변수는 변경되지 않음.
print(x.data)

1.0


In [4]:
x.data = np.array(2.) # Variable 클래스를 상자로 이용
print(x.data)

2.0


In [5]:
import numpy as np
x = np.array(1)
x.ndim



0

In [6]:
x = np.array([1, 2, 3])
x.ndim

1

In [7]:
x = np.array([[1, 2, 3], 
             [4, 5, 6]])  # 2차원 배열, 대괄호 2개 임에 유의!
x.ndim

2

# STEP2.

In [8]:
class Config:
  enable_backprop = True

In [9]:
import weakref

class Function:
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)  # 별표를 붙여 언팩
    if not isinstance(ys, tuple):  # 튜플이 아닌 경우 튜플로 변경
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    if Config.enable_backprop:
      self.generation = max([x.generation for x in inputs])  # 입력 변수와 같은 세대 값으로 설정. 입력 변수가 둘 이상이라면 가장 큰 generation의 수 선택
      for output in outputs:
        output.set_creator(self)  # 출력 변수에 창조자를 설정한다. -> 이 부분이 연결을 동적으로 만드는 기법의 핵심(DeZero의 동적 계산 그래프)
      self.inputs = inputs  # s6. 역전파 시 활용하기 위해 입력 변수 기억(보관)한다.
  #   self.outputs = outputs  # 출력도 저장한다.
      self.outputs = [weakref.ref(output) for output in outputs] # 순환참조 문제 해결을 위해 약한 참조로 수정(메모리 사용량 증가x)

    return outputs if len(outputs) > 1 else outputs[0]  # 리스트의 원소가 하나라면 첫 번째 원소를 반환한다.

  def forward(self, xs):
    raise NotImplementedError()

  def backward(self, gys):  # s6. 역전파로 확장
    raise NotImplementedError()

In [10]:
class Square(Function):
  def forward(self, x):
    y = x ** 2
    return y
  
  def backward(self, gy): # 역전파 메서드 추가확장
    x = self.inputs[0].data # Function 클래스의 인스턴스 변수 이름이 단수형인 input에서 복수형인 inputs로 변경 된 것에 따른 수정
    gx = 2 * x * gy
    return gx

In [11]:
'''
x = Variable(np.array(10))
f = Square() # 기존 f = Function() 에서 square() 함수로 세분화한 것에 따른 수정.
y = f(x)

print(type(y))
print(y.data)
'''

'\nx = Variable(np.array(10))\nf = Square() # 기존 f = Function() 에서 square() 함수로 세분화한 것에 따른 수정.\ny = f(x)\n\nprint(type(y))\nprint(y.data)\n'

# STEP3.

In [12]:
class Exp(Function):
  def forward(self, x):
    y = np.exp(x)
    return y

  def backward(self, gy):
    x = self.input.data
    gx = np.exp(x) * gy
    return gx

In [13]:
'''
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
# print(y.data)

y.grad = np.array(1.)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)
'''

'\nA = Square()\nB = Exp()\nC = Square()\n\nx = Variable(np.array(0.5))\na = A(x)\nb = B(a)\ny = C(b)\n# print(y.data)\n\ny.grad = np.array(1.)\nb.grad = C.backward(y.grad)\na.grad = B.backward(b.grad)\nx.grad = A.backward(a.grad)\nprint(x.grad)\n'

# STEP4.

In [14]:
def numerical_diff(f, x, eps=1e-4):
  x0 = Variable(x.data - eps)
  x1 = Variable(x.data + eps)
  y0 = f(x0)
  y1 = f(x1)
  return (y1.data - y0.data) / (2 * eps)


In [15]:
'''
f = Square()
x = Variable(np.array(2.))
dy = numerical_diff(f, x)
print(dy)
'''

'\nf = Square()\nx = Variable(np.array(2.))\ndy = numerical_diff(f, x)\nprint(dy)\n'

In [16]:
'''
def f(x):
  A = Square()
  B = Exp()
  C = Square()
  return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)
'''

'\ndef f(x):\n  A = Square()\n  B = Exp()\n  C = Square()\n  return C(B(A(x)))\n\nx = Variable(np.array(0.5))\ndy = numerical_diff(f, x)\nprint(dy)\n'

# STEP6. 

# STEP7.

In [17]:
'''
# s7에서 수정된 Variable을 이용하여 역전파가 자동으로 실행됨을 확인.

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
# print(y.data)

y.grad = np.array(1.)
y.backward()
# b.grad = C.backward(y.grad)
# a.grad = B.backward(b.grad)
# x.grad = A.backward(a.grad)
print(x.grad)
'''

'\n# s7에서 수정된 Variable을 이용하여 역전파가 자동으로 실행됨을 확인.\n\nA = Square()\nB = Exp()\nC = Square()\n\nx = Variable(np.array(0.5))\na = A(x)\nb = B(a)\ny = C(b)\n# print(y.data)\n\ny.grad = np.array(1.)\ny.backward()\n# b.grad = C.backward(y.grad)\n# a.grad = B.backward(b.grad)\n# x.grad = A.backward(a.grad)\nprint(x.grad)\n'

# STEP8

# STEP9 함수를 더 편리하게

In [18]:
def square(x): # Square, Exp를 "class"로 구현해서 인스턴스 생성 및 인스턴스 호출 두 단계로 코딩해야 하는 불편함을 개선하기 위해 "함수"로 구현
  f = Square()
  return f(x)

def exp(x):
  f = Exp()
  return f(x)

In [19]:
'''
x = Variable(np.array(0.5))
y = square(exp(square(x)))  # 연속하여 적용도 가능
y.grad = np.array(1.)
y.backward()
print(x.grad)
'''

'\nx = Variable(np.array(0.5))\ny = square(exp(square(x)))  # 연속하여 적용도 가능\ny.grad = np.array(1.)\ny.backward()\nprint(x.grad)\n'

In [20]:
'''
x = Variable(np.array(0.5))
y = square(exp(square(x)))  # 연속하여 적용도 가능

y.backward()
print(x.grad)
'''

'\nx = Variable(np.array(0.5))\ny = square(exp(square(x)))  # 연속하여 적용도 가능\n\ny.backward()\nprint(x.grad)\n'

In [21]:
'''
x = Variable(np.array(1.))
x = Variable(None)

x = Variable(1.)

'''

'\nx = Variable(np.array(1.))\nx = Variable(None)\n\nx = Variable(1.)\n\n'

In [22]:
x = np.array([1.0])
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'>


In [23]:
x = np.array(1.0)
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


In [24]:
def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return(x)

# STEP10 테스트

코랩에서 unittest 사용하는 방법을 모르겠음..
터미널 명령어를 어떻게 실행해야 하는지,
test 파일을 어느 경로에 둬야 하는지

# STEP11 가변 길이 인수(순전파 편)


In [25]:
'''
class Add(Function):
  def forward(self, xs):
    x0, x1 = xs
    y = x0 + x1
    return (y,)
'''    

'\nclass Add(Function):\n  def forward(self, xs):\n    x0, x1 = xs\n    y = x0 + x1\n    return (y,)\n'

In [26]:
'''
xs = [Variable(np.array(2)), Variable(np.array(3))]  # 리스트로 준비
f = Add()
ys = f(xs)  # 튜플
y = ys[0]
print(y.data)
'''

'\nxs = [Variable(np.array(2)), Variable(np.array(3))]  # 리스트로 준비\nf = Add()\nys = f(xs)  # 튜플\ny = ys[0]\nprint(y.data)\n'

# STEP12 가변 길이 인수(개선 편)

In [27]:
'''
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)
'''

'\nx0 = Variable(np.array(2))\nx1 = Variable(np.array(3))\nf = Add()\ny = f(x0, x1)\nprint(y.data)\n'

In [28]:
class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y

In [29]:
def add(x0, x1):  # Add 클래스를 파이썬 함수로 사용할 수 있게
  return Add()(x0, x1)

In [30]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)  # Add 클래스 생성 과정이 감춰짐
print(y.data)

5


# STEP13 가변 길이 인수(역전파 편)

In [31]:
class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y

  def backward(self, gy):
    return gy, gy

In [32]:
x = Variable(np.array(2.))
y = Variable(np.array(3.))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


# STEP14 같은 변수 반복 사용

In [33]:
"""
# 첫 번째 계산
x = Variable(np.array(3.))
y = add(x, x)
y.backward()
print(x.grad)

# 두 번째 계산(같은 x를 사용하여 다른 계산을 수행)
x.cleargrad() # 미분값 초기화
y = add(add(x, x), x)

y.backward()
print(x.grad)
"""

'\n# 첫 번째 계산\nx = Variable(np.array(3.))\ny = add(x, x)\ny.backward()\nprint(x.grad)\n\n# 두 번째 계산(같은 x를 사용하여 다른 계산을 수행)\nx.cleargrad() # 미분값 초기화\ny = add(add(x, x), x)\n\ny.backward()\nprint(x.grad)\n'

# STEP15-16 복잡한 계산그래프(이론 편, 구현 편)

In [34]:
"""
generations = [2, 0, 1, 4, 2]
funcs = []

for g in generations:
  f = Function()
  f.generation = g
  funcs.append(f)

[f.generation for f in funcs]"""

'\ngenerations = [2, 0, 1, 4, 2]\nfuncs = []\n\nfor g in generations:\n  f = Function()\n  f.generation = g\n  funcs.append(f)\n\n[f.generation for f in funcs]'

In [35]:
"""
funcs.sort(key=lambda x: x.generation)
[f.generation for f in funcs]

f = funcs.pop()
f.generation"""

'\nfuncs.sort(key=lambda x: x.generation)\n[f.generation for f in funcs]\n\nf = funcs.pop()\nf.generation'

In [36]:
"""
x = Variable(np.array(2.))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)"""

'\nx = Variable(np.array(2.))\na = square(x)\ny = add(square(a), square(a))\ny.backward()\n\nprint(y.data)\nprint(x.grad)'

# STEP17 메모리 관리와 순환 참조

# STEP18 메모리 절약 모드

In [37]:
x0 = Variable(np.array(1.))
x1 = Variable(np.array(1.))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

None None
2.0 1.0


In [38]:
import contextlib

@contextlib.contextmanager
def using_config(name, value):
  old_value = getattr(Config, name)
  setattr(Config, name, value)
  try:
    yield
  finally:
    setattr(Config, name, old_value)

In [39]:
def no_grad():
  return using_config('enable_backprop', False)

with no_grad():
  x = Variable(np.array(2.))
  y = square(x)

# STEP19 변수 사용성 개선

In [40]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
print(x.shape)

(2, 3)


In [42]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
print(len(x))

2


In [49]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
print(x)

variable([[1 2 3]
          [4 5 6]])
